#Project_sim

In [ ]:
# -*- coding: utf-8 -*-
#files required to run: 
#universal_sentence_vectorizer, tf_idf_vectorizer, project_list, project_features
from universal_sentence_vectorizer import ContentVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tf_idf_vectorizer import tf_idf_model
import pandas as pd
import json
from sqlalchemy import create_engine


class project_sim(object):
    def __init__(self):
        with open('project_list.json') as json_file:
            project_list = json.load(json_file)
        with open('project_features.json') as json_file:
            temp_project_dic = json.load(json_file)
        project_features = pd.DataFrame(temp_project_dic)   
        self.embeddingmodel = ContentVectorizer()
        self.tf_idfmodel = tf_idf_model()
        self.tf_idfmodel.pca()
        
    def tf_idf_embedding(self, pid):
        return self.tf_idfmodel.embedding[str(pid)]
        
    def embedding(self, pid):
        with open('project_features.json') as json_file:
            temp_project_dic = json.load(json_file)
        project_features = pd.DataFrame(temp_project_dic)      
        return self.embeddingmodel.encode_content(project_features[project_features['index']==pid][['title','description']].to_dict('records')[0])
    
    def cossin_similarity(self, vec1, vec2):
        return cosine_similarity([vec1, vec2])[0][1] * 100

    def generate_project_simlarity(self):
        with open('project_list.json') as json_file:
            project_list = json.load(json_file)
        with open('project_features.json') as json_file:
            temp_project_dic = json.load(json_file)
        project_features = pd.DataFrame(temp_project_dic)
        project_list_keylist = []
        for key in project_list.keys():
            project_list_keylist.append(int(key))
        results = []
        count = 0
        for project1 in project_list_keylist:
            encode = []
            tfidf = []
            for project2 in project_list_keylist:
                if project1 != project2:
                    encode.append({'pid':project2,'sim_score':self.cossin_similarity(self.embedding(project1),self.embedding(project2))})
                    tf_title_score = self.cossin_similarity(self.tf_idfmodel.embedding[str(project1)]['vec_title'], self.tf_idfmodel.embedding[str(project2)]['vec_title'])
                    tf_des_score = self.cossin_similarity(self.tf_idfmodel.embedding[str(project1)]['vec_des'], self.tf_idfmodel.embedding[str(project2)]['vec_des'])
                    tfidf.append({'pid':project2,'sim_score':0.7*tf_title_score+0.3*tf_des_score})
            encode = sorted(encode, key = lambda x: x['sim_score'], reverse = True)[:20]
            tfidf = sorted(tfidf, key = lambda x: x['sim_score'], reverse = True)[:20]
            for i in range(20):
                results.append({'pid':int(project1), 
                                'encode_sim':int(encode[i]['pid']),
                                'encode_sim_score': encode[i]['sim_score'],
                                'tfidf_sim':int(tfidf[i]['pid']),
                                'tfidf_sim_score': tfidf[i]['sim_score']})
            count += 1
            print (count)
        project_sim_table = pd.DataFrame(results)
        project_sim_table.to_csv('project_sim_table.csv',index=False)  
    
        return project_sim_table
    
if __name__ == '__main__':
    x = project_sim()
    sim_results = x.generate_project_simlarity()

#data mask

##upload project

In [ ]:
# -*- coding: utf-8 -*-
import json
import pandas as pd
with open('project_list.json') as json_file:
    project_list = json.load(json_file)
with open('project_features.json') as json_file:
    temp_project_dic = json.load(json_file)
project_features = pd.DataFrame(temp_project_dic)  

In [ ]:
import random
publisher_id = []
for i in range(200):
  publisher_id.append(random.randrange(1, 40000))
#upload_project = project_features['index']
upload_project = project_features[['index']].copy()
upload_project.rename(columns = {'index':'project_id'}, inplace = True)
upload_project['publisher_id'] = publisher_id
upload_project

In [ ]:
upload_project.to_csv('upload_project.csv',index=False)  

##user sim table fake

In [ ]:
#pip install DataSynthesizer

In [ ]:
from DataSynthesizer.DataDescriber import DataDescriber

from DataSynthesizer.DataGenerator import DataGenerator

from DataSynthesizer.ModelInspector import ModelInspector

from DataSynthesizer.lib.utils import read_json_file, display_bayesian_network
input_data ='user_sim_table.csv'
mode = 'independent_attribute_mode'
description_file = 'description.json'
synthetic_data = 'sythetic_data.csv'

In [ ]:
threshold_value = 20 

# Number of tuples generated in synthetic dataset.
num_tuples_to_generate = 50000 # Here 32561 is the same as input dataset, but it can be set to another number.
describer = DataDescriber(category_threshold=threshold_value)
describer.describe_dataset_in_random_mode(input_data)
describer.save_dataset_description_to_file(description_file)

In [ ]:
import pandas as pd

# Read both datasets using Pandas.
input_df = pd.read_csv(input_data, skipinitialspace=True)
synthetic_df = pd.read_csv(synthetic_data)
# Read attribute description from the dataset description file.
attribute_description = read_json_file(description_file)['attribute_description']

inspector = ModelInspector(input_df, synthetic_df, attribute_description)

In [ ]:
sim_score_df = synthetic_df[['sim_score']]
import random
from itertools import repeat
import numpy as np
user_id_ls =random.sample(range(1,2000), 1000)
user_id =  [x for item in user_id_ls for x in repeat(item, 50)]
user_id.sort()
sim_user = np.random.choice(user_id_ls,size=len(user_id)).tolist()
user_sim_df = pd.DataFrame(user_id, columns = ['user_id'])
user_sim_df['sim_user'] = sim_user
user_sim_df['sim_score'] = sim_score_df['sim_score']
user_sim_df.to_csv('user_sim_table.csv',index=False)  

##top click

In [ ]:
import pandas as pd
top_click = pd.read_csv('top_click_all.csv')
node_list = top_click['node'].tolist()
#count number of node in each user
node_count = []
len(node_list)
for item in node_list:
  node_count.append(item.count('n'))
max_node= max(node_count)
#count_node_list
node_list_count = [i for i in range(0,max_node+1)]
repetition_count = []
for i in range(max_node+1):
  repetition_count.append(node_count.count(i))
sum(repetition_count)
node_stats_df = pd.DataFrame(node_list_count, columns= ['num_node_each_user'])
node_stats_df['count'] = repetition_count
count_perc = []
for i in range(max_node+1):
  count_perc.append(round(repetition_count[i]/len(node_count)*100,2))
node_stats_df['percentage'] = count_perc

In [ ]:
import re
extrat_node = []
for item in node_list:
  extrat_node.append([int(extrat_node) for extrat_node in re.findall(r'-?\d+\.?\d*', item)])
single_node_list = []
for item in extrat_node:
  for node in item:
    single_node_list.append(node)
res = {}
for i in single_node_list:
    res[i] = single_node_list.count(i)
node_usage = list(res.values())

In [ ]:
print("max: " + str(max(node_usage)))
import statistics
print("mean: " + str(statistics.mean(node_usage)))
less_five_count = 0
five_ten_count = 0
ten_twenty_count = 0
twenty_thirty_count = 0
greater_thirty = 0
for item in node_usage:
  if item <= 5:
    less_five_count +=1;
  elif item > 5 and item <= 10:
    five_ten_count += 1;
  elif item > 10 and item <= 20:
    ten_twenty_count += 1;
  elif item > 20 and item <= 30:
    twenty_thirty_count += 1;
  else:
    greater_thirty += 1
print("number of project used less than 5: " + str(less_five_count))
print("number of project used between 5 and 10: " + str(five_ten_count))
print("number of project used between 10 and 20: " + str(ten_twenty_count))
print("number of project used between 20 and 30: " + str(twenty_thirty_count))
print("number of project used greater 30: " + str(greater_thirty))
#最多的一个project用了39次，平均使用率是1.7

In [ ]:
import json
import random
with open('project_features.json') as json_file:
    temp_project_dic = json.load(json_file)
project_features = pd.DataFrame(temp_project_dic)
project_num_list = project_features['index'].tolist()
project_num_list
#assume the first one is used repeatly 35 times
for i in range (35):
  project_num_list.append(project_num_list[0])
#assume the second,third one is used repeatly 25 times
for i in range (25):
  project_num_list.append(project_num_list[1])
  project_num_list.append(project_num_list[2])
#assume the forth one is used repeatly 16 times
for i in range (16):
  project_num_list.append(project_num_list[3])
#assume the fifth one is used repeatly 8 times
for i in range (8):
  project_num_list.append(project_num_list[4])
#assume the sixth one is used repeatly 6 times
for i in range (6):
  project_num_list.append(project_num_list[5])
#assume the 7~14 one is used repeatly 3 times
for i in range (3):
  project_num_list.append(project_num_list[6])
  project_num_list.append(project_num_list[7])
  project_num_list.append(project_num_list[8])
  project_num_list.append(project_num_list[9])
  project_num_list.append(project_num_list[10])
  project_num_list.append(project_num_list[11])
  project_num_list.append(project_num_list[12])
  project_num_list.append(project_num_list[13])
  project_num_list.append(project_num_list[14])
#assume the 14~24 one is used repeatly 2 times
for i in range (2):
  project_num_list.append(project_num_list[15])
  project_num_list.append(project_num_list[16])
  project_num_list.append(project_num_list[17])
  project_num_list.append(project_num_list[18])
  project_num_list.append(project_num_list[19])
  project_num_list.append(project_num_list[20])
  project_num_list.append(project_num_list[21])
  project_num_list.append(project_num_list[22])
  project_num_list.append(project_num_list[23])
  project_num_list.append(project_num_list[24])
random.seed(10)
random.shuffle(project_num_list)
import numpy as np
user_sim_id = pd.read_csv("user_sim_table.csv")
user_sim_id_list = user_sim_id['user_id'].tolist()
user_sim_id_uni = np.unique(user_sim_id_list).tolist()
project_number_list = ['None'] * len(user_sim_id_uni)
# 7% of users have 1 project. 7%*1000 = 70
for i in range(70):
  project_number_list[i] = 1
# 3% of users have 2 project. 3%*1000 = 30
for i in range(70,100):
  project_number_list[i] = 2
# 1% of users have 3 project. 1%*1000 = 10
for i in range(100,110):
  project_number_list[i] = 3
# 1% of users have 4 project. 1%*1000 = 10
for i in range(110,120):
  project_number_list[i] = 4
#5~11 projects only 1 person
for i in range(120,124):
  project_number_list[i] = i-115
for i in range(len(project_number_list)):
  if project_number_list[i] == 'None':
    project_number_list[i] = 0
random.shuffle(project_number_list)
len(project_number_list)
uid_project_id_match = []
for i in range(len(user_sim_id_uni)):
  uid_project_id_match.append(np.random.choice(project_num_list, project_number_list[i], replace=False).tolist())
uid_project_id_match

In [ ]:
node = []
for i in range(len(uid_project_id_match)):
  node_each = "["
  if len(uid_project_id_match[i]) == 0:
    node_each += "]"
  else:
    for j in range (len(uid_project_id_match[i])-1):
      node_each += "'/node/"
      node_each += str(uid_project_id_match[i][j])
      node_each += "',"
    node_each += "'/node/"
    node_each += str(uid_project_id_match[i][len(uid_project_id_match[i])-1])
    node_each += "']"
  node.append(node_each)

In [ ]:
top_click_all_pd = pd.DataFrame(user_sim_id_uni, columns = ['uid'])
top_click_all_pd['node'] = node
top_click_all_pd.to_csv('top_click_all.csv',index=False)  

## view score

In [ ]:
import json
import random

with open('project_list.json') as json_file:
  project_dict = json.load(json_file)
project_key = list(project_dict.keys())
view_score = []
for i in range(len(project_key)):
  view_score.append(random.random())
view_score

project_dict_view = {}
for key in project_key:
    for value in view_score:
        project_dict_view[key] = value
        view_score.remove(value)
        break  
project_dict_view

with open("project_dict_view.json", "w") as outfile:
    json.dump(project_dict_view, outfile)

#project_recom

##package

In [ ]:
#pip install paramiko

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 212 kB 4.1 MB/s 
     |████████████████████████████████| 856 kB 52.8 MB/s 
     |████████████████████████████████| 4.1 MB 45.6 MB/s 
     |████████████████████████████████| 62 kB 1.1 MB/s 


In [ ]:
#pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 1.6 MB/s 


In [ ]:
#pip install mysql-connector-python-rf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.9 MB 3.9 MB/s 
  Created wheel for mysql-connector-python-rf: filename=mysql_connector_python_rf-2.2.2-cp37-cp37m-linux_x86_64.whl size=249476 sha256=976f3b90cb58039a6e4f6e5411ae845565981a2c0374dcb5ed3aeb7406af9098
  Stored in directory: /root/.cache/pip/wheels/68/59/cf/3b03557b26b4c75af3788a553e0ff9cf0b37a22d0c9cb01979
Successfully built mysql-connector-python-rf


##create_df

In [ ]:
# -*- coding: utf-8 -*-
import time
import pandas as pd
import pickle
import paramiko
import pymysql
import json




class project_recom(object):
    def __init__(self):
        with open('project_dict_view.json') as json_file:
          self.project_dict = json.load(json_file)
 #      self.project_dict = load_sim_table_from_MySQL()[0]
        self.click_table = pd.read_csv('top_click_all.csv')
        self.user_sim_table = pd.read_csv('user_sim_table.csv')
        self.project_sim_table = pd.read_csv('project_sim_table.csv')
        self.upload_project = pd.read_csv('upload_project.csv')
        self.user_list = list(set(self.user_sim_table['user_id'].tolist()))
 #       self.user_list = load_user_list()

    def recom_score(self, uid, project_id):
        view_score = self.project_dict[str(project_id)]
        encode_score = 0
        history = self.get_project_num(self.click_table[self.click_table['uid']==uid])
        history.extend(self.upload_project[self.upload_project['publisher_id']==uid]['project_id'].values) #add uploaded project num
        if len(history) != 0:
            sim_score_data=pd.DataFrame(None)
            for project in history:
                #switch, search for project most similar to project_id
                sim_score_temp = self.project_sim_table[(self.project_sim_table['pid']==int(project))&(self.project_sim_table['encode_sim']==project_id)]
                sim_score_data = pd.concat([sim_score_data,sim_score_temp])
                sim_score_temp_2 = self.project_sim_table[(self.project_sim_table['pid']==int(project_id))&(self.project_sim_table['encode_sim']==project)]
                sim_score_data = pd.concat([sim_score_data,sim_score_temp])
            if len(sim_score_data)>0:
                encode_score = float(sim_score_data.sort_values(by='encode_sim_score', ascending=False)['encode_sim_score'].values[0])/100
        
        total_score = float(view_score)*0.5 + float(encode_score)*0.5
        return total_score

    def get_userbased_project_list(self, uid, n_near, m_recent):
        sim_user_list = self.user_sim_table[self.user_sim_table['user_id']==uid]['sim_user'][0:n_near]
        project_list = []
        if len(sim_user_list) != 0:
            for sim_user in sim_user_list:
                project_list.extend(self.get_project_num(self.click_table[self.click_table['uid']==sim_user])[:m_recent])
        
        project_list = list(set(project_list))
        return project_list
    
    def get_contentbased_project_list(self, uid, n_near, m_recent):
        history = self.get_project_num(self.click_table[self.click_table['uid']==uid])[:m_recent]
        history.extend(self.upload_project[self.upload_project['publisher_id']==uid]['project_id'].values) #add uploaded project num
        project_list = []
        if len(history) != 0:
            for pid in history:
                project_list.extend(self.project_sim_table[self.project_sim_table['pid']==int(pid)]['encode_sim'][0:n_near])
                
        
        project_list = list(set(project_list))
        return project_list
                
    
 
    def update_single_user(self, uid):
        usr_project_list = self.get_userbased_project_list(uid, 9, 9)
        cont_project_list = self.get_contentbased_project_list(uid, 9, 9)
        project_list = list(set(usr_project_list+cont_project_list))
        project_recom_df = pd.DataFrame(None)
        if len(project_list)!=0:
            project_recom_list = []
            for elem in project_list:
                if str(elem) in list(self.project_dict): #make sure they are in the project table
                    project_recom_list.append({'user': uid, 'project': elem,'rec_score':self.recom_score(uid,elem) })
            project_recom_df = pd.DataFrame(project_recom_list)
            if len(project_recom_df)>0: #in case there are no projects applicable
                project_recom_df = project_recom_df.sort_values(by='rec_score',ascending=False)
            project_recom_df = project_recom_df.reset_index(drop=True)
        return project_recom_df    

    def update_all_user(self):
        project_rec_table = pd.DataFrame(None)
        for uid in self.user_list:
            project_rec_temp = self.update_single_user(uid)
            print(uid)
            project_rec_table = pd.concat([project_rec_table, project_rec_temp], ignore_index=True)
        return project_rec_table


    def get_project_num(self,data):
        num_list = []
        if len(data)==0:
            return num_list   
        clean_data = data['node'].values[0].replace('[','').replace(']','').replace("'","")
        if len(clean_data)==0:
            return num_list
        for node in clean_data.split(','):
            num_list.append(node.split('/')[-1])
        return num_list
    
    
    

if __name__ == '__main__':
    x = project_recom()
    #recom = x.update_whole_table()
    start = time.time()
    #recom1 = x.get_userbased_project_list(488)
    #recom2 = x.get_contentbased_project_list(99)
    recom = x.update_single_user(488)
    #print (time.time()-start)


##recom_df

In [ ]:
# -*- coding: utf-8 -*-
project_recom_user = project_recom()
non_empty_user = project_recom_user.update_all_user()

In [ ]:
non_empty_user

In [ ]:
project_dic = project_recom_user.project_dict
project_dict_view = project_recom_user.project_dict
project_dict_view = (sorted(project_dict_view.items(), key=lambda item: item[1], reverse=True))
#{0: 0, 2: 1, 1: 2, 4: 3, 3: 4}#(sorted(x.items(), key=lambda item: item[1]))
pop_list = []
for i in range(10):
  pop_list.append(int(project_dict_view[i][0]))
pop_list

[815, 432, 1017, 1522, 646, 1975, 1674, 458, 314, 902]

In [ ]:
non_empty_list = non_empty_user['user'].tolist()
user_sim_table = pd.read_csv('user_sim_table.csv')
user_list = list(set(user_sim_table['user_id'].tolist()))

In [ ]:
import random
gen_recom_list = []
for i in range (len(user_list)):
  if user_list[i] not in  non_empty_list:
    random_list = random.sample(pop_list, 5)
    for j in range (5):
      gen_recom = []
      gen_recom.append(user_list[i])
      gen_recom.append(random_list[j])
      gen_recom.append(0.5*project_dic[str(random_list[j])])
      gen_recom_list.append(gen_recom)
gen_recom_list

In [ ]:
recom_list = non_empty_user.values.tolist()
from pandas.core import missing
from collections import Counter
recom_list_exist = []
for i in range (len(recom_list)):
  recom_list_exist.append(recom_list[i][0])
count_recom = dict(Counter(recom_list_exist))
missing_dic = {}
for item in count_recom.keys():
  if count_recom[item] < 5:
    missing_dic[item] = 5-count_recom[item]

In [ ]:
missing_list = list(map(list, missing_dic.items()))
recom_fill = []
for i in range (len(missing_list)):
  for j in range (missing_list[i][1]):
      random_list = random.sample(pop_list, missing_list[i][1])
      gen_recom = []
      gen_recom.append(missing_list[i][0])
      gen_recom.append(random_list[j])
      gen_recom.append(0.5*project_dic[str(random_list[j])])
      recom_fill.append(gen_recom)
gen_recom

[1996, 1017, 0.4935689211441361]

In [ ]:
total_list = recom_list + gen_recom_list + recom_fill

In [ ]:
recom_df_total = pd.DataFrame(sorted(total_list, key=lambda x: x[0]))
recom_df_total.columns = ['user','project','recom_score']

In [ ]:
recom_df_total= recom_df_total.sort_values(by=['user', 'recom_score'])
recom_df_total

,user,project,recom_score
2,1,1334,0.181500
1,1,1785,0.320949
0,1,562,0.385483
4,1,432,0.497119
3,1,815,0.497993
...,...,...,...
6685,1998,902,0.462465
6683,1998,314,0.464192
6684,1998,1674,0.478222
6681,1998,1017,0.493569


In [ ]:
recom_df_total.to_csv('recommendation.csv')